In [1]:
import pandas as pd
import numpy as np
import os
import pickle

# explore the data

import argparse
import os
import json
from typing import List, Tuple
from loguru import logger
from ehrshot.utils import LABELING_FUNCTION_2_PAPER_NAME
import pandas as pd
from tqdm import tqdm
from femr.datasets import PatientDatabase
import femr
from datetime import datetime
from collections import defaultdict

In [2]:
# one line of code to configure the patient database
def configure_database(path_to_database: str) -> PatientDatabase:
    return PatientDatabase(path_to_database)

In [3]:
df_anemia = pd.read_csv('EHRSHOT_ASSETS/benchmark/lab_anemia/labeled_patients.csv')
df_hypoglycemia = pd.read_csv('EHRSHOT_ASSETS/benchmark/lab_hypoglycemia/labeled_patients.csv')
df_hyponatremia = pd.read_csv('EHRSHOT_ASSETS/benchmark/lab_hyponatremia/labeled_patients.csv')
df_thrombocytopenia = pd.read_csv('EHRSHOT_ASSETS/benchmark/lab_thrombocytopenia/labeled_patients.csv')
df_hyperkalemia = pd.read_csv('EHRSHOT_ASSETS/benchmark/lab_hyperkalemia/labeled_patients.csv')

In [4]:
df_anemia = df_anemia.drop(columns=['label_type'])
df_hypoglycemia = df_hypoglycemia.drop(columns=['label_type'])
df_hyponatremia = df_hyponatremia.drop(columns=['label_type'])
df_thrombocytopenia = df_thrombocytopenia.drop(columns=['label_type'])
df_hyperkalemia = df_hyperkalemia.drop(columns=['label_type'])

# Renaming 'value' column for clarity
df_anemia = df_anemia.rename(columns={'value': 'value_anemia'})
df_hypoglycemia = df_hypoglycemia.rename(columns={'value': 'value_hypoglycemia'})
df_hyponatremia = df_hyponatremia.rename(columns={'value': 'value_hyponatremia'})
df_thrombocytopenia = df_thrombocytopenia.rename(columns={'value': 'value_thrombocytopenia'})
df_hyperkalemia = df_hyperkalemia.rename(columns={'value': 'value_hyperkalemia'})

# Merging the dataframes on 'prediction_time' and 'patient_id'
df_merged = pd.merge(df_anemia, df_hypoglycemia, on=['prediction_time', 'patient_id'], how='inner')
df_merged = pd.merge(df_merged, df_hyponatremia, on=['prediction_time', 'patient_id'], how='inner')
df_merged = pd.merge(df_merged, df_thrombocytopenia, on=['prediction_time', 'patient_id'], how='inner')
df_merged = pd.merge(df_merged, df_hyperkalemia, on=['prediction_time', 'patient_id'], how='inner')

In [5]:
columns_to_check = ['value_anemia', 'value_hypoglycemia', 'value_hyponatremia',
                    'value_thrombocytopenia', 'value_hyperkalemia']
df_filtered = df_merged[
    df_merged[columns_to_check].apply(lambda x: (x != 0).sum(), axis=1) >= 1
]

In [8]:
df_merged

,patient_id,prediction_time,value_anemia,value_hypoglycemia,value_hyponatremia,value_thrombocytopenia,value_hyperkalemia
0,115973769,2011-10-01 11:19:00,0,0,0,0,0
1,115973769,2011-10-05 14:24:00,0,0,0,0,0
2,115973769,2011-10-06 02:59:00,1,0,0,0,0
3,115973769,2011-10-08 04:49:00,0,0,0,1,0
4,115973769,2012-08-24 14:44:00,0,0,0,0,0
...,...,...,...,...,...,...,...
152326,115967121,2023-02-10 09:23:00,2,0,0,0,0
152327,115967121,2023-02-12 08:44:00,2,0,0,0,0
152328,115967121,2023-02-16 10:45:00,2,0,0,0,0
152329,115967121,2023-02-23 09:17:00,2,0,0,0,0


In [10]:
df_merged_copy = df_merged.copy()
columns_to_modify = ['value_anemia', 'value_hypoglycemia', 'value_hyponatremia',
                     'value_thrombocytopenia', 'value_hyperkalemia']
df_merged_copy[columns_to_modify] = df_merged_copy[columns_to_modify].where(df_merged_copy[columns_to_modify] <= 0, 1)
# df_merged_copy

,patient_id,prediction_time,value_anemia,value_hypoglycemia,value_hyponatremia,value_thrombocytopenia,value_hyperkalemia
0,115973769,2011-10-01 11:19:00,0,0,0,0,0
1,115973769,2011-10-05 14:24:00,0,0,0,0,0
2,115973769,2011-10-06 02:59:00,1,0,0,0,0
3,115973769,2011-10-08 04:49:00,0,0,0,1,0
4,115973769,2012-08-24 14:44:00,0,0,0,0,0
...,...,...,...,...,...,...,...
152326,115967121,2023-02-10 09:23:00,1,0,0,0,0
152327,115967121,2023-02-12 08:44:00,1,0,0,0,0
152328,115967121,2023-02-16 10:45:00,1,0,0,0,0
152329,115967121,2023-02-23 09:17:00,1,0,0,0,0


In [13]:
df_merged_copy.to_csv('white_box_common_data/lab_test.csv', index=False)

In [11]:
df_merged

,patient_id,prediction_time,value_anemia,value_hypoglycemia,value_hyponatremia,value_thrombocytopenia,value_hyperkalemia
0,115973769,2011-10-01 11:19:00,0,0,0,0,0
1,115973769,2011-10-05 14:24:00,0,0,0,0,0
2,115973769,2011-10-06 02:59:00,1,0,0,0,0
3,115973769,2011-10-08 04:49:00,0,0,0,1,0
4,115973769,2012-08-24 14:44:00,0,0,0,0,0
...,...,...,...,...,...,...,...
152326,115967121,2023-02-10 09:23:00,2,0,0,0,0
152327,115967121,2023-02-12 08:44:00,2,0,0,0,0
152328,115967121,2023-02-16 10:45:00,2,0,0,0,0
152329,115967121,2023-02-23 09:17:00,2,0,0,0,0


In [88]:
# sampled_df = df_filtered.groupby('patient_id').apply(lambda x: x.sample(n=1)).reset_index(drop=True)
sampled_df = df_filtered

In [89]:
columns_to_modify = ['value_anemia', 'value_hypoglycemia', 'value_hyponatremia',
                     'value_thrombocytopenia', 'value_hyperkalemia']
sampled_df[columns_to_check] = sampled_df[columns_to_check].where(sampled_df[columns_to_check] <= 0, 1)

/tmp/ipykernel_136288/1732081118.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df[columns_to_check] = sampled_df[columns_to_check].where(sampled_df[columns_to_check] <= 0, 1)


In [90]:
sampled_df[columns_to_check].mean()

value_anemia              0.862692
value_hypoglycemia        0.019146
value_hyponatremia        0.345930
value_thrombocytopenia    0.407362
value_hyperkalemia        0.023157
dtype: float64

In [91]:
percentage_criteria = 20

# Calculate the number of non-zero values required per row to meet the percentage criteria
num_columns = 2  # Since we are only considering two columns
min_nonzero_required = num_columns * (percentage_criteria / 100)

filtered_df_2 = sampled_df[
    sampled_df[['value_hypoglycemia', 'value_hyperkalemia']].apply(lambda x: (x == 1).sum(), axis=1) >= min_nonzero_required
]

In [92]:
filtered_df_2[columns_to_modify].mean()

value_anemia              0.700079
value_hypoglycemia        0.458645
value_hyponatremia        0.382237
value_thrombocytopenia    0.294210
value_hyperkalemia        0.554746
dtype: float64

In [98]:
# filtered_df_final = filtered_df_2.sample(128, random_state=97)
# filtered_df_final[columns_to_modify].mean()
filtered_df_final = filtered_df_2.sample(n=100, random_state=100)
filtered_df_final = filtered_df_final.reset_index(drop=True)

In [2]:
# filtered_df_final.to_csv('lab_test_patient.csv', index=False)